In [24]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import ADASYN
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble  import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
!pip install lightgbm
import lightgbm as lgb
import numpy as np
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
from tabulate import tabulate
import pickle

In [25]:
le=LabelEncoder()

In [26]:
np.random.seed(30)

In [27]:
df=pd.read_csv("data.csv")

In [28]:
df=df.drop(['age_group', 'bmi_category',
       'year_of_birth', 'diabetes'],axis=1)

In [29]:
df['gender']=le.fit_transform(df['gender'])
df['hypertension']=le.fit_transform(df['hypertension'])
df['heart_disease']=le.fit_transform(df['heart_disease'])
df['ever_married']=le.fit_transform(df['ever_married'])
df['work_type']=le.fit_transform(df['work_type'])
df['smoking_status']=le.fit_transform(df['smoking_status'])
df['stroke']=le.fit_transform(df['stroke'])
df['type_of_residence']=le.fit_transform(df['type_of_residence'])

In [30]:
X=df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'type_of_residence', 'avg_glucose_level', 'bmi',
       'smoking_status']]
Y=df['stroke']

In [31]:
ad=ADASYN(sampling_strategy='minority',random_state=32)
x_ros1,y_ros1=ad.fit_resample(X,Y)


In [32]:
x_train1,x_test1,y_train1,y_test1=train_test_split(X,Y,test_size=0.25,random_state=30)
x_train2,x_test2,y_train2,y_test2=train_test_split(X,Y,test_size=0.20,random_state=30)

In [33]:
dtc=DecisionTreeClassifier(criterion='gini',max_depth=9,random_state=30)
adadtc=AdaBoostClassifier(dtc)
model21=adadtc.fit(x_train1,y_train1)
etc=ExtraTreesClassifier(random_state=30)
model27=etc.fit(x_train1,y_train1)
x=xgb.XGBClassifier(learning_rate= 1, max_depth=6, min_child_weight=1,random_state=30)
model31=x.fit(x_train1,y_train1)

In [34]:
v1=VotingClassifier(estimators=[('model 21',model21),('model 31',model31)],voting="hard")
v2=VotingClassifier(estimators=[('model 31',model31),('model 27',model27)],voting="hard")
vc3hard=VotingClassifier(estimators=[('v1',v1),('v2',v2)],voting="hard")
model41=vc3hard.fit(x_train1,y_train1)

In [35]:
model=model41

In [36]:
y_pred41=model41.predict(x_test1)

In [37]:
for i in range(0,2000):
  if y_pred41[i]==1:
    print(x_test1.iloc[i])
    print(y_pred41[i])
    print(y_test1.iloc[i])

gender                1.0
age                  71.0
hypertension          1.0
heart_disease         0.0
ever_married          1.0
work_type             4.0
type_of_residence     0.0
avg_glucose_level    87.8
bmi                  30.8
smoking_status        3.0
Name: 44878, dtype: float64
1
1
gender                 0.00
age                   72.00
hypertension           0.00
heart_disease          0.00
ever_married           1.00
work_type              3.00
type_of_residence      1.00
avg_glucose_level    219.91
bmi                   32.10
smoking_status         3.00
Name: 44858, dtype: float64
1
1
gender                 0.000000
age                   75.000000
hypertension           0.000000
heart_disease          1.000000
ever_married           0.000000
work_type              4.000000
type_of_residence      1.000000
avg_glucose_level    109.780000
bmi                   28.318273
smoking_status         3.000000
Name: 45047, dtype: float64
1
1
gender                 1.00
age             

In [38]:
print("Voting Best of 3 Hard,Odd Numbered Models 25% Test",accuracy_score(y_test1,y_pred41),f1_score(y_test1,y_pred41),precision_score(y_test1,y_pred41),recall_score(y_test1,y_pred41)
   )

Voting Best of 3 Hard,Odd Numbered Models 25% Test 0.9806827202963747 0.12048192771084336 1.0 0.0641025641025641


In [39]:
with open('model.pkl', 'wb') as model_file:
  pickle.dump(model, model_file)

In [40]:
! pip install -q gradio
with open('model.pkl','rb') as file:
  model=pickle.load(file)

In [41]:
def bmi_category(num):
  if num<=18.5:
    return "Underweight"
  elif num>18.5 and num<=24.9:
    return "Healthy Weight"
  elif num>25.0 and num<29.9:
    return "Overweight"
  else:
    return "Obese"
def age_year_convert(num):
  return (2020-num)
def age_group(age):
  if(age>=0 and age<=2):
    return "Baby"
  elif(age>=3 and age<=39):
    return "Young Adult"
  elif(age>=40 and age<=59):
    return "Middle Aged Adult"
  else:
    return "Old Adult"
def diabetes_check(num):
  if num<=100.0:
    return "Normal"
  elif num>100.0 and num<125.0:
    return "Prediabetes"
  else:
    return "Diabetes"

In [42]:
le_gender = {'Male': 1, 'Female':0 , 'Other': 2}
le_hypertension = {'No':0 , 'Yes':1}
le_heart_disease = {'Yes': 1, 'No': 0}
le_ever_married = {'Yes': 1, 'No': 0}
le_work_type = {'Children':0, 'Govt Job':1, 'Private':2, 'Never Worked':3, 'Self Employed':4}
le_smoking_status = {'Currently Smokes':0, 'Formerly Smoked':1,  'Never Smoked':2, 'Unknown Status':3}
le_age_group = {'Young Adult': 3, 'Middle Aged Adult': 2, 'Old Adult': 1, 'Baby': 0}
le_bmi_category = {'Healthy Weight':0,'Obese':1,'Overweight':2,'Underweight':3}
le_diabetes = {'Diabetes':0,'Prediabetes':1,'Normal':2}
le_type_of_residence = {'Urban': 1, 'Rural': 0}

In [43]:
def prediction(location,gender,age,hypertension,heart_disease,ever_married,work_type,type_of_residence,avg_glucose_level,bmi,smoking_status):
  list1=[le_gender[gender]]
  list2=[age]
  list3=[le_hypertension[hypertension]]
  list4=[le_heart_disease[heart_disease]]
  list5=[le_ever_married[ever_married]]
  list6=[le_work_type[work_type]]
  list7=[le_type_of_residence[type_of_residence]]
  list8=[avg_glucose_level]
  list9=[bmi]

  list10=[le_smoking_status[smoking_status]]

  # agegrp=age_group(age)
  # list11=[le_age_group[agegrp]]
  # bmicat=bmi_category(bmi)
  # list12=[le_bmi_category[bmicat]]
  # ageyear=age_year_convert(age)
  # list13=[ageyear]
  # dia=diabetes_check(avg_glucose_level)
  # list14=[le_diabetes[dia]]
  df=pd.DataFrame(list(zip(list1,list2,list3,list4,list5,list6,list7,list8,list9,list10)), columns=['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'type_of_residence', 'avg_glucose_level', 'bmi',
       'smoking_status'])
  pred=model.predict(df)[0]
  final_msg=""
  maps=""
  if pred==0:
    final_msg= "No Brain Stroke Detected"
  else:
    final_msg="Brain Stroke Detected"
  if(pred==1):
    maps_url = f"https://www.google.com/maps/search/hospitals+near+{location}"
    maps= f"Find nearby hospitals: [Google Maps]({maps_url})"
  else:
    maps="Visiting a Hospital is not required"
  return final_msg,maps






In [44]:
print(prediction("malakpet","Male",71,"Yes","No","Yes","Self Employed","Rural",87.8,30.8,"Unknown Status"))

('Brain Stroke Detected', 'Find nearby hospitals: [Google Maps](https://www.google.com/maps/search/hospitals+near+malakpet)')


In [45]:
! pip install -q gradio
import gradio as gr

In [47]:
interface=gr.Interface(
    fn=prediction,
    title="Brain Stroke Prediction with Textual Inputs using Ensemble Machine Learning",
    inputs=[
        gr.Textbox(label="Location"),
        gr.Radio(choices=["Male","Female","Other"],label="Gender"),
        gr.Slider(0,100,label="Age"),
        gr.Radio(choices=["No","Yes"],label="Presence of Hypertension"),
        gr.Radio(choices=["No","Yes"],label="Presence of Heart Disease"),
        gr.Radio(choices=["No","Yes"],label="Ever Married"),
        gr.Radio(choices=["Children","Govt Job","Private","Never Worked","Self Employed"],label="Work Type"),
        gr.Radio(choices=["Urban","Rural"],label="Type of Residence"),
        gr.Slider(0,300,label="Average Glucose Level"),
        gr.Slider(0,100,label="Body Mass Index in kg/m^2"),
        gr.Radio(choices=["Currently Smokes","Formerly Smoked","Never Smoked","Unknown Status"],label="Smoking Status")
    ],
    outputs=[
        gr.Textbox(label="Prediction by Model"),
        gr.Markdown(label="Maps")
    ])

interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://adb7bb00e292b3bf9a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
